# EE4375-2022: Second Lab Session: Matrix Build 

## Import Packages  

In [1]:
using LinearAlgebra 
using SparseArrays 

using IterativeSolvers
using Preconditioners

using BenchmarkTools
using Profile
using ProfileView

using Plots 

## Section 1: Build Linear System as Sparse From the Start

In [2]:
function buildMat1D(N::Int64)
  Nm1::Int64 = N-1; Np1::Int64 = N+1 
  h::Float64 = 1/N; h2::Float64 = h*h; 
  stencil = Vector{Float64}([-1/h2, 2/h2, -1/h2]);
  #..Allocate row, column and value vector 
  I = Int64[]; sizehint!(I, 3*Nm1); 
  J = Int64[]; sizehint!(J, 3*Nm1); 
  vals=Float64[]; sizehint!(vals, 3*Nm1);
  intervalRows = Vector{Int64}(2:N);
  for i in intervalRows 
    append!(I, [i,i,i])
    append!(J, [i-1,i,i+1])
    append!(vals,stencil) 
  end 
  #..Build matrix for interior rows   
  A = sparse(I,J,vals,Np1,Np1)
  #..Build matrix for boundary rows
  A[1,1] = 1; A[end,end]=1; A[2,1] =0; A[end-1,end]=0; 
  return A 
end 

buildMat1D (generic function with 1 method)

In [3]:
buildMat1D(4)

5×5 SparseMatrixCSC{Float64, Int64} with 11 stored entries:
 1.0     ⋅      ⋅      ⋅    ⋅ 
 0.0   32.0  -16.0     ⋅    ⋅ 
  ⋅   -16.0   32.0  -16.0   ⋅ 
  ⋅      ⋅   -16.0   32.0  0.0
  ⋅      ⋅      ⋅      ⋅   1.0

In [4]:
@code_warntype buildMat1D(5);

MethodInstance for buildMat1D(::Int64)
  from buildMat1D(N::Int64) in Main at In[2]:1
Arguments
  #self#::Core.Const(buildMat1D)
  N::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  A::SparseMatrixCSC{Float64, Int64}
  intervalRows::Vector{Int64}
  vals::Vector{Float64}
  J::Vector{Int64}
  I::Vector{Int64}
  stencil::Vector{Float64}
  h2::Float64
  h::Float64
  Np1::Int64
  Nm1::Int64
  i::Int64
Body::SparseMatrixCSC{Float64, Int64}
1 ─       Core.NewvarNode(:(A))
│   %2  = (N - 1)::Int64
│   %3  = Base.convert(Main.Int64, %2)::Int64
│         (Nm1 = Core.typeassert(%3, Main.Int64))
│   %5  = (N + 1)::Int64
│   %6  = Base.convert(Main.Int64, %5)::Int64
│         (Np1 = Core.typeassert(%6, Main.Int64))
│   %8  = (1 / N)::Float64
│   %9  = Base.convert(Main.Float64, %8)::Float64
│         (h = Core.typeassert(%9, Main.Float64))
│   %11 = (h * h)::Float64
│   %12 = Base.convert(Main.Float64, %11)::Float64
│         (h2 = Core.typeassert(%12, Main.Float64))
│   %14 = Core.apply_

In [18]:
A = buildMat1D(4);
typeof(A)

SparseMatrixCSC{Float64, Int64}